<a href="https://colab.research.google.com/github/prateek-jd/Python/blob/master/1_IMDBDatabase_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB Dataset Assignment
#### For this assignment I am having an sqlite3 database file, which contains records of Movies released from 1931 to 2018 with deatils of its cast, country, ratings, director, producer and genre information in multiple relational tables.

#### This data is asusual, not a perfect dataset. before using this I cleaned & preprocessed this data just to avoid repeating of codes for each question. This assignment is to check SQL skills, so I am going to make it with most of SQL Queries only.

### **Task 1**: List all Directors, who directed comedy movie in leap year.

---



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
# Connect with the database
import pandas as pd
import sqlite3 as sq

con = sq.connect('/content/drive/My Drive/Colab Notebooks/Db-IMDB.db')

In [2]:
# Get data
data1 = pd.read_sql_query('''SELECT DISTINCT
(M_Director.PID) as Director_ID,
   trim(Person.Name) as Director 	-- Selecting Columns having Director ID and their names.
FROM
   M_Director 
   LEFT JOIN
      Movie USING (MID) 		-- Joining tables (M_Director with Movie) to link director id's to Movie.
   JOIN
      M_Genre USING (MID) 		-- Joining table above joined table with M_Genre(Table) to link Movie with their Genre.
   JOIN
      Person USING (PID) 		-- Joining above with Person to get names of Director using their PID.
WHERE
   M_Genre.GID in 
   (
      SELECT
         Genre.GID 
      FROM
         Genre 
      WHERE
         Genre.Name like '%Comedy%'  -- Filtering above data for Comedy genre.
   )
   AND substr(Movie.year, length(Movie.year) - 3, 4) % 4 = 0 	-- Filtering  above for leap year.''',con)
print('Output 1: Following are director names, who directed comedy movies in leap year.\n')
data1

Output 1: Following are director names, who directed comedy movies in leap year.



,Director_ID,Director
0,nm1179457,Milap Zaveri
1,nm0500444,Danny Leiner
2,nm0440604,Anurag Kashyap
3,nm0178997,Frank Coraci
4,nm0001162,Griffin Dunne
...,...,...
187,nm0474891,Siddharth Anand Kumar
188,nm2490004,Amma Rajasekhar
189,nm0667393,Oliver Paulus
190,nm4110102,Raja Chanda


### **Task 2**: List the names of all the actors who played in the movie 'Anand' (1971)

---



In [3]:
# Get data
data2 = pd.read_sql_query('''SELECT DISTINCT
(M_Cast.PID) as Cast_ID,
   trim(Person.Name) as [Cast]	-- Selecting Cast id and cast name as output.
FROM
   Person 
   LEFT JOIN
      M_Cast USING (PID)	-- Joining tables (person with M_Cast) to link Cast id with Cast Name.
WHERE
   M_Cast.MID = 
   (
      SELECT
         MID 
      FROM
         Movie 
      WHERE
         title = 'Anand'	-- Filtering data for Anand Movie.
   )''',con)
print('Output 2: Following are Cast names, who who acted in movie Anand(2017).\n')
data2

Output 2: Following are Cast names, who who acted in movie Anand(2017).



,Cast_ID,Cast
0,nm0004435,Rajesh Khanna
1,nm0000821,Amitabh Bachchan
2,nm0764407,Sumita Sanyal
3,nm0219946,Ramesh Deo
4,nm0219947,Seema Deo
5,nm0783996,Asit Kumar Sen
6,nm0222743,Dev Kishan
7,nm0695155,Atam Prakash
8,nm0474931,Lalita Kumari
9,nm1783998,Savita


### **Task 3**: List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

---




In [4]:
# Get data
data3 = pd.read_sql_query('''SELECT DISTINCT(Person.PID) as Cast_ID, Person.Name as Cast_Name	-- Selecting Cast ID and Name as output field
FROM Person
JOIN M_Cast USING (PID)	-- Joining M_Cast with Person to get Names according to ID	
WHERE M_Cast.MID IN
	(
	SELECT DISTINCT(Movie.MID)
	FROM Movie
	WHERE substr(Movie.year, length(Movie.year)-3,4) < 1970	-- Filtering data for condition 'year before 1970'
	OR
	substr(Movie.year, length(Movie.year)-3,4) > 1990	-- Filtering data for condition 'year after 1990' 		
	)''',con)
print('Output 3: Following are Cast names, who who acted in movies before 1970 and after 1990.\n')
data3

Output 3: Following are Cast names, who who acted in movies before 1970 and after 1990.



,Cast_ID,Cast_Name
0,nm0000288,Christian Bale
1,nm0000949,Cate Blanchett
2,nm1212722,Benedict Cumberbatch
3,nm0365140,Naomie Harris
4,nm0785227,Andy Serkis
...,...,...
32514,nm2675737,Nirvasha Jithoo
32515,nm2370589,Kamal Maharshi
32516,nm1866356,Mohini Manik
32517,nm0066829,Iqbal


### **Task 4**: List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

---



In [5]:
# Get data
data4 = pd.read_sql_query('''SELECT DISTINCT (TRIM(Person.Name)) AS Director, -- Selecting Director Name and number of movies as output field
 Movies
FROM Person -- Joining Person table to below query result to link names to the resultant director Id

JOIN
  (SELECT *
   FROM
     (SELECT DISTINCT (M_Director.PID) AS Director_ID,
                      COUNT(DISTINCT(M_Director.MID)) AS Movies
      FROM M_Director
      GROUP BY Director_ID -- Getting table of Director Id and number of directed movies
 )
   WHERE Movies >= 10 -- Filtering Directors who directed 10 or more movies
 ) ON Person.PID = Director_ID
ORDER BY Movies DESC -- Ordering list in descending order on number of movies''',con)
print('Output 4: Following is list of directors directed 10 or more movies.\n')
data4

Output 4: Following is list of directors directed 10 or more movies.



,Director,Movies
0,David Dhawan,39
1,Mahesh Bhatt,35
2,Ram Gopal Varma,30
3,Priyadarshan,30
4,Vikram Bhatt,29
5,Hrishikesh Mukherjee,27
6,Yash Chopra,21
7,Shakti Samanta,19
8,Basu Chatterjee,19
9,Subhash Ghai,18


### **Task 5a**: For each year, count the number of movies in that year that had only female actors.

---



In [6]:
# Get data
data5a = pd.read_sql_query('''SELECT DISTINCT
(SUBSTR(MOVIE.YEAR, LENGTH(MOVIE.YEAR) - 3, 4)) AS M_YEAR,
   IFNULL(MOVIES, 0) AS F_MCOUNT 	-- Selecting Year and Movie count as output field
FROM
   MOVIE 
   LEFT JOIN
      -- Joining Movie with resultant table to get year which doesn't have any movie with only female cast.
(
      SELECT
         SUBSTR(MOVIE.YEAR, LENGTH(MOVIE.YEAR) - 3, 4) AS M_YEAR, COUNT(DISTINCT(MOVIE.MID)) AS MOVIES 
      FROM
         MOVIE 
      WHERE
         MOVIE.MID NOT IN 
         (
            SELECT
(M_CAST.MID) 
            FROM
               M_CAST 
               JOIN
                  PERSON USING (PID) 						-- Filtering movies out of the list which having male characters
            WHERE
               PERSON.GENDER = 'Male' 
         )
         AND MOVIE.MID IN 
         (
            SELECT
(M_CAST.MID) 
            FROM
               M_CAST 
               JOIN
                  PERSON USING (PID) 						-- Filtering movies which having Female characters
            WHERE
               PERSON.GENDER = 'Female' 
         )
      GROUP BY
         M_YEAR) 			-- By combination of above 2 filters we will get Movies having only female characters.
         ON M_YEAR = SUBSTR(MOVIE.YEAR, LENGTH(MOVIE.YEAR) - 3, 4)''',con)
print('Output 5a: Following is the list of years having movies with only female cast:')
data5a

Output 5a: Following is the list of years having movies with only female cast:


,M_YEAR,F_MCOUNT
0,2018,1
1,2012,0
2,2016,0
3,2017,0
4,2008,0
...,...,...
73,1947,0
74,1936,0
75,1946,0
76,1943,0


#### Another Solution to Task 5a

---



In [7]:
# Faster Solution
data5a1 = pd.read_sql_query('''select distinct(substr(movie.year,length(movie.year)-3,4)) as M_Year, ifnull(Movies,0) as F_MCount
FROM Movie
left JOIN
(
select Film_Year, count(Film_Name) as Movies from
(
SELECT Film_Year, Film_Name, count(Male_Cast) as Male_CastSize,count(Female_Cast) as Female_CastSize from
(
SELECT	substr(movie.year,length(movie.year)-3,4) as Film_Year, M_Cast.MID as Film_ID,Movie.title AS Film_Name,
		CASE
			WHEN Person.Gender = 'Male' THEN (M_Cast.PID)
		END AS Male_Cast,
		CASE
			WHEN Person.Gender = 'Female' THEN (M_Cast.PID)
		END AS Female_Cast
FROM	M_Cast
JOIN 	Movie	--Joining M_Cast and Movie table to get movie name and cast count.
USING	(MID)
JOIN	Person
USING 	(PID)
)
group by Film_name
)
WHERE Male_CastSize = 0 AND Female_CastSize > 0
GROUP by film_year
)
on M_Year = Film_Year
order by M_Year ASC''',con)
data5a1

,M_Year,F_MCount
0,1931,0
1,1936,0
2,1939,1
3,1941,0
4,1943,0
...,...,...
73,2014,0
74,2015,0
75,2016,0
76,2017,0


### **Task 5b**: Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year.

---



In [8]:
# Get data
data5b = pd.read_sql_query('''SELECT M_YEAR, 
       MCOUNT, 
       Ifnull(F_MCOUNT, 0) AS Female_Mcount, 
       Ifnull(( ( F_MCOUNT * 100.00 ) / MCOUNT )|| '%', 0)   AS F_Percent	-- Selecting output fields
FROM   
	(
	SELECT 	Substr(YEAR, Length(YEAR) - 3, 4) AS M_Year, 
			Count(DISTINCT( MID )) AS MCount	 -- Selecting table having year wise movie count 
	FROM	MOVIE 
    GROUP	BY M_YEAR
	) 
LEFT JOIN 	-- Joining with table having movie count which have only female cast year wise
	(
	SELECT 	Substr(MOVIE.YEAR, Length(MOVIE.YEAR) - 3, 4) AS F_Year,
			Count(DISTINCT( MOVIE.MID )) AS F_MCount 
	FROM	MOVIE 
	WHERE	MOVIE.MID NOT IN
			(
			SELECT	M_CAST.MID 
			FROM	M_CAST 
			JOIN	PERSON using (PID) 
			WHERE	PERSON.GENDER = 'Male'
			) 
		AND	MOVIE.MID IN
			(
			SELECT	M_CAST.MID 
            FROM	M_CAST 
			JOIN	PERSON using (PID) 
			WHERE  PERSON.GENDER = 'Female'
			) 
		GROUP  BY	F_YEAR
		) 
	ON 	M_YEAR = F_YEAR ''',con)
print('Output 5b: Following is the list of years having movies with only female cast:')
data5b

Output 5b: Following is the list of years having movies with only female cast:


,M_Year,MCount,Female_Mcount,F_Percent
0,1931,1,0,0
1,1936,3,0,0
2,1939,2,1,50.0%
3,1941,1,0,0
4,1943,1,0,0
...,...,...,...,...
73,2014,126,0,0
74,2015,119,0,0
75,2016,129,0,0
76,2017,126,0,0


### **Task 6a**: Find the film(s) with the largest cast. Return the movie title and the size of the cast.

---


##### By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

---



In [9]:
data6a = pd.read_sql_query('''SELECT	trim(Movie.title) as Film_Name,
		count(distinct(PID)) as Cast_Size 
FROM	M_Cast
JOIN 	Movie	--Joining M_Cast and Movie table to get movie name and cast count.
USING	(MID)
GROUP BY	Film_Name
ORDER BY	Cast_size DESC''',con)
print('Output 6a: following is the list of movies name with number of cast.')
data6a

Output 6a: following is the list of movies name with number of cast.


,Film_Name,Cast_Size
0,Ocean's Eight,238
1,Apaharan,233
2,Gold,215
3,My Name Is Khan,213
4,Captain America: Civil War,191
...,...,...
3339,Return of Hanuman,1
3340,Kala Jigar,1
3341,Chaar Sahibzaade 2: Rise of Banda Singh Bahadur,1
3342,Yeh Hai Malegaon Ka Superman,0


### **Task 6b**: In addition to Task 6a, show cast size bifurcated w.r.t gender.

---




In [10]:
data6b = pd.read_sql_query('''SELECT Film_Name, count(Male_Cast) as Male_CastSize,count(Female_Cast) as Female_CastSize from
(
SELECT	M_Cast.MID as Film_ID,Movie.title AS Film_Name,
		CASE
			WHEN Person.Gender = 'Male' THEN (M_Cast.PID)
		END AS Male_Cast,
		CASE
			WHEN Person.Gender = 'Female' THEN (M_Cast.PID)
		END AS Female_Cast
FROM	M_Cast
JOIN 	Movie	--Joining M_Cast and Movie table to get movie name and cast count.
USING	(MID)
JOIN	Person
USING 	(PID)
)
group by Film_name''',con)
print('Output 6b: following is the list of movies name with number of cast as per gender bifurcation.')
data6b

Output 6b: following is the list of movies name with number of cast as per gender bifurcation.


,Film_Name,Male_CastSize,Female_CastSize
0,'D',30,2
1,'Kaash',20,4
2,...Aur Pyaar Ho Gaya,10,9
3,...Yahaan,8,5
4,100 Days,11,4
...,...,...,...
3337,Zor: Never Underestimate the Force,23,7
3338,Zubaan,13,4
3339,Zubeidaa,14,9
3340,Zulm Ki Hukumat,16,7


### **Task 7**: Find the decade D with the largest number of films and the total number of films in D.

---


#####A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on.

---



In [11]:
#Get Data
data7 = pd.read_sql_query('''SELECT (d_year||' - '||(d_year +9)) AS Decade,
       sum(mcount) AS M_Count
FROM
  (SELECT distinct(cast((substr(YEAR, length(YEAR)-3, 4)) AS INTEGER)) AS d_year
   FROM movie
   ORDER BY d_year)
LEFT JOIN
  (SELECT cast(substr(YEAR, length(YEAR)-3, 4) AS INTEGER) AS M_year,
          count(distinct(MID)) AS MCount
   FROM Movie
   GROUP BY m_year
   ORDER BY M_year) ON d_year <= M_year
AND m_year < d_year + 10
GROUP BY decade''',con)
print('Output 7: Following is the outcome:\n')
data7

Output 7: Following is the outcome:



,Decade,M_Count
0,1931 - 1940,6
1,1936 - 1945,7
2,1939 - 1948,11
3,1941 - 1950,14
4,1943 - 1952,25
...,...,...
73,2014 - 2023,604
74,2015 - 2024,478
75,2016 - 2025,359
76,2017 - 2026,230


### **Task 8**: Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies).

---



To use SQLite Window function, I need to update SQLite to 3.31 or greater in Colab
Following code will update SQLite to the latest availabe version.

---


!add-apt-repository -y ppa:dqlite/stable # Update Sqlite

!apt update

!apt install sqlite3

---


Restart Kernal/Runtime


In [13]:
#Get required Data
data8 = pd.read_sql_query('''SELECT 	DISTINCT(M_Cast.PID) as ID,
		trim(Person.Name) as Cast_Name
FROM	M_Cast
JOIN	Person USING (PID)
WHERE	ID NOT IN
(
SELECT 	DISTINCT(Cast_ID) as UEC_ID
FROM
(
SELECT 	*,
		lead(M_year,1,0) OVER (PARTITION BY Cast_ID ORDER BY M_Year ASC) as N_Year
FROM
(
SELECT 	ifnull(M_Cast.PID,'<Unknown>') as Cast_ID,
		CAST(substr(Movie.year,length(Movie.year)-3,4) as INTEGER) as M_Year
FROM	M_Cast
JOIN	Movie
USING	(MID)
)
)
WHERE (N_Year-M_Year) > 3
)''',con)
print('Output 8: Following is the list of actors never unemployed more than 3 years.(Included actors having single movie):\n')
data8

Output 8: Following is the list of actors never unemployed more than 3 years.(Included actors having single movie):



,ID,Cast_Name
0,nm0000288,Christian Bale
1,nm0000949,Cate Blanchett
2,nm1212722,Benedict Cumberbatch
3,nm0365140,Naomie Harris
4,nm0785227,Andy Serkis
...,...,...
32122,nm2371614,Hayley Cleghorn
32123,nm2675737,Nirvasha Jithoo
32124,nm2370589,Kamal Maharshi
32125,nm1866356,Mohini Manik


### **Task 9**: Find all the actors that made more movies with Yash Chopra than any other director.

---



In [14]:
#Get required Data
data9 = pd.read_sql_query('''SELECT Actor_ID,trim(Person.Name) as Actor,Film
FROM
(
SELECT M_Cast.PID as Actor_ID, M_Director.PID as Director_ID, count(distinct(MID)) as Film, rank() OVER (PARTITION by M_Cast.PID ORDER by count(distinct(MID)) desc) as F_Rank
FROM M_Cast
JOIN M_Director USING (MID)
group by Actor_ID,Director_ID
)
JOIN Person on Actor_ID = Person.PID
WHERE Director_ID = (SELECT PID FROM Person WHERE Person.Name like '%Yash Chopra%') AND F_Rank = 1
GROUP BY Actor_ID
Order by Film desc''',con)
print('Output 9: Following is the list of actors that made more movies with Yash Chopra than any other director:\n')
data9

Output 9: Following is the list of actors that made more movies with Yash Chopra than any other director:



,Actor_ID,Actor,Film
0,nm0707271,Jagdish Raj,11
1,nm0471443,Manmohan Krishna,10
2,nm0407002,Iftekhar,9
3,nm0004434,Shashi Kapoor,7
4,nm0716851,Waheeda Rehman,5
...,...,...,...
240,nm0080266,Raj Bharti,1
241,nm0052570,Saira Banu,1
242,nm0046230,Badri Prasad,1
243,nm0019463,Murad Ali,1


### **Task 10**: Return all actors whose Shahrukh number is 2.

---


#####The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc.

---



In [15]:
#Get data from Query (It takes 22298ms on my PC)
data10 = pd.read_sql_query('''SELECT DISTINCT(M_Cast.PID) as Actor2_ID,Person.Name as Actor2
FROM
(
SELECT distinct(Actor1_ID) as Actor1_PID,M_Cast.MID as Movie1_ID
FROM
(
SELECT Movie_ID,M_Cast.PID as Actor1_ID
FROM
(
SELECT M_cast.mid as Movie_ID
FROM M_Cast
WHERE M_Cast.PID = (Select person.PID from Person where person.name like '%Shah Rukh Khan%')
)
LEFT JOIN M_Cast ON M_Cast.mid = Movie_ID
)
LEFT JOIN M_Cast on M_Cast.PID = Actor1_PID
)
Left JOIN M_Cast on Movie1_ID = M_Cast.MID
JOIN Person on M_cast.PID = person.PID
order by actor2_id''',con)
print('Output 10: Following is the list of actors with Shahrukh number 2:\n')
data10

Output 10: Following is the list of actors with Shahrukh number 2:



,Actor2_ID,Actor2
0,nm0000027,Alec Guinness
1,nm0000047,Sophia Loren
2,nm0000093,Brad Pitt
3,nm0000096,Gillian Anderson
4,nm0000112,Pierce Brosnan
...,...,...
28438,nm9984769,Fernando Cruz
28439,nm9984770,Sohail Mirza
28440,nm9985086,Shreyas Sanghavi
28441,nm9988016,Ashiqa Salvan


#### Another Solution to Task 10

---

In [16]:
#Get data from Query (It takes 8538ms on my PC)
data101 = pd.read_sql_query('''SELECT distinct(M_Cast.PID) as Actor2_ID, Person.Name as Actor2
FROM M_Cast
left JOIN Person on Actor2_ID = Person.PID
WHERE M_Cast.MID in
(
SELECT M_Cast.MID as Movie1_ID
FROM M_Cast
WHERE M_cast.PID in
(
SELECT M_cast.PID as Actor1_ID 
FROM M_Cast 
WHERE M_Cast.MID in
(
SELECT M_cast.MID as Movie_ID
FROM
M_Cast WHERE M_Cast.PID = (Select person.PID from Person where person.name like '%Shah Rukh Khan')
)
)
)
order by Actor2_id''',con)
data101

,Actor2_ID,Actor2
0,nm0000027,Alec Guinness
1,nm0000047,Sophia Loren
2,nm0000093,Brad Pitt
3,nm0000096,Gillian Anderson
4,nm0000112,Pierce Brosnan
...,...,...
28438,nm9984769,Fernando Cruz
28439,nm9984770,Sohail Mirza
28440,nm9985086,Shreyas Sanghavi
28441,nm9988016,Ashiqa Salvan
